In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
import GenDBScraper.Utilities.nb_utilities as nbu
import logging
import ipywidgets as widgets
from GenDBScraper.PseudomonasDotComScraper import PseudomonasDotComScraper, pdc_query
import pandas

In [3]:
from pymongo import MongoClient

In [4]:
uri = "localhost:27017"

#The MongoClient constructor accepts many different arguments to configure how the driver connects to MongoDB and how many operations will be performed. We'll look at the most basic configuration first, which is passing the SRV string of our Atlas cluster to MongoClient.

client = MongoClient(uri)

In [5]:
sbw25_pdc = client.sbw25_pdc

In [6]:
logger = logging.getLogger()

In [7]:
logger.setLevel(logging.INFO)

In [8]:
sbw25_pdc = client["sbw25_pdc"]

In [9]:
# Load table of features in PDC.
feature_table = pandas.read_csv('/home/grotec/data/sbw25/features.csv')

In [29]:
feature_table[feature_table["Locus Tag"]=="PFLU2154"]

,Strain,Assembly Status,Replicon,Locus Tag,RefSeq Accession,GI Number,UniProtKB Accession,UniProtKB ID,Uniparc,UniRef100 ID,...,Feature Type,Coordinates,Gene Length,Gene Name,Product Description,Product Name Confidence,Estimated MW (kDa),Estimated Isoelectric Point (pI),Estimated Charge (ph7),Unnamed: 24
2114,Pseudomonas fluorescens SBW25,Complete Genome|GCF_000009225.2|latest,chromosome,PFLU2154,YP_002871770.1,229589651.0,C3K787,C3K787_PSEFS,UPI00019D9EBF,UniRef100_C3K787,...,CDS,2333683..2334012(-),330,NaN,hypothetical protein,NaN,12.6,5.61,-3.11,NaN


In [11]:
features = [v for v in feature_table.transpose().to_dict().values()]

In [12]:
features[0]

{'Strain': 'Pseudomonas fluorescens SBW25',
 'Assembly Status': 'Complete Genome|GCF_000009225.2|latest',
 'Replicon': 'chromosome',
 'Locus Tag': 'PFLU0001',
 'RefSeq Accession': 'YP_002869698.1',
 'GI Number': 229587579.0,
 'UniProtKB Accession': nan,
 'UniProtKB ID': nan,
 'Uniparc': nan,
 'UniRef100 ID': nan,
 'UniRef90 ID': nan,
 'UniRef50 ID': nan,
 'NCBI Locus Tag': nan,
 'NCBI Old Locus Tag': nan,
 'Affymetrix': nan,
 'Feature Type': 'CDS',
 'Coordinates': '1..1506(+)',
 'Gene Length': 1506,
 'Gene Name': 'dnaA',
 'Product Description': 'chromosomal replication initiation protein',
 'Product Name Confidence': nan,
 'Estimated MW (kDa)': 56.3,
 'Estimated Isoelectric Point (pI)': 8.88,
 'Estimated Charge (ph7)': 6.33,
 'Unnamed: 24': nan}

In [13]:
for feature in features: 
    feature["tag"] = ["SBW25", feature["Locus Tag"]]

In [14]:
feature_collection = sbw25_pdc["features"]

In [31]:
feature_collection.insert_many(features)

In [15]:
strain="SBW25"

In [21]:
locus_tags=feature_table["Locus Tag"].values

In [ ]:
results = {}

In [30]:
for locus_tag in locus_tags[2114:]:
    logging.info("Running %s", locus_tag)


    query=pdc_query(strain=strain, feature=locus_tag)
    
    print(query)
    pdc = PseudomonasDotComScraper(query)
    
    pdc.connect()
    try:
        pdc_functions = pdc._get_functions_pathways_go(pdc._get_feature_url(query))
        
    except Exception as exc:
        raise
        logging.error("Data retrieval for query %s failed. Error log follows.", query)
        logging.error("Error type: %s", type(exc) )
        logging.error("Error message: %s", str(exc))
        
        continue

    id_str = "__".join([query.strain, query.feature])
    
    functions = [v for v in pdc_functions['Functional Predictions from Interpro'].values()]
    if len(functions) > 0:
        for function in functions:
            function["tag"] = [strain, locus_tag]
        function_insert_response = sbw25_pdc.functions.insert_many(functions)
  
    ontologies = [v for v in pdc_functions['Gene Ontology'].values()]
    if ontologies is not None and len(ontologies) > 0:
        for item in ontologies:
            item["tag"] = [strain, locus_tag]
        ontology_insert_response = sbw25_pdc.ontologies.insert_many(ontologies)
    
    results[id_str] = {
        "pcd_functions": pdc_functions,
        "function_insert_response": function_insert_response,
        "ontology_insert_response": ontology_insert_response,
    }

2020-01-24 07:39:02,579 INFO: Running PFLU2154


pdc_query(strain='SBW25', feature='PFLU2154', organism=None)


2020-01-24 07:39:03,804 INFO: Connected to https://www.pseudomonas.com .
2020-01-24 07:39:04,899 INFO: Connected to https://www.pseudomonas.com/primarySequenceFeature/list?c1=name&v1=PFLU2154&e1=1&term1=SBW25&assembly=complete .
2020-01-24 07:39:05,942 INFO: Connected to https://www.pseudomonas.com/feature/show?id=1462328&view=functions .
2020-01-24 07:39:05,980 INFO: No Interpro Data found on https://www.pseudomonas.com/feature/show?id=1462328&view=functions
2020-01-24 07:39:05,982 INFO: Running PFLU2155


pdc_query(strain='SBW25', feature='PFLU2155', organism=None)


2020-01-24 07:39:06,666 INFO: Connected to https://www.pseudomonas.com .
2020-01-24 07:39:07,493 INFO: Connected to https://www.pseudomonas.com/primarySequenceFeature/list?c1=name&v1=PFLU2155&e1=1&term1=SBW25&assembly=complete .
2020-01-24 07:39:08,255 INFO: Connected to https://www.pseudomonas.com/feature/show?id=1462330&view=functions .
2020-01-24 07:39:08,310 INFO: Running PFLU2156


...

2020-01-24 11:03:36,424 INFO: Connected to https://www.pseudomonas.com .
2020-01-24 11:03:37,158 INFO: Connected to https://www.pseudomonas.com/primarySequenceFeature/list?c1=name&v1=PFLU6134&e1=1&term1=SBW25&assembly=complete .
2020-01-24 11:03:37,910 INFO: Connected to https://www.pseudomonas.com/feature/show?id=1470186&view=functions .
2020-01-24 11:03:37,968 INFO: Running PFLU6135


pdc_query(strain='SBW25', feature='PFLU6135', organism=None)


2020-01-24 11:03:38,658 INFO: Connected to https://www.pseudomonas.com .
2020-01-24 11:03:39,389 INFO: Connected to https://www.pseudomonas.com/primarySequenceFeature/list?c1=name&v1=PFLU6135&e1=1&term1=SBW25&assembly=complete .
2020-01-24 11:03:40,139 INFO: Connected to https://www.pseudomonas.com/feature/show?id=1470188&view=functions .
2020-01-24 11:03:40,188 INFO: Running PFLU6136


pdc_query(strain='SBW25', feature='PFLU6136', organism=None)


2020-01-24 11:03:40,874 INFO: Connected to https://www.pseudomonas.com .
2020-01-24 11:03:41,610 INFO: Connected to https://www.pseudomonas.com/primarySequenceFeature/list?c1=name&v1=PFLU6136&e1=1&term1=SBW25&assembly=complete .
2020-01-24 11:03:42,361 INFO: Connected to https://www.pseudomonas.com/feature/show?id=1470190&view=functions .


In [20]:
results

{'SBW25__PFLU0001': {'pcd_functions': {'Gene Ontology': {0: {'Ontology': 'Biological Process',
     'Accession': 'GO:0006270',
     'Term': 'DNA replication initiation',
     'GO Evidence': 'ISM Inferred from Sequence Model  Term mapped from: InterPro:PS01008',
     'Evidence Ontology (ECO) Code': 'ECO:0000259 match to InterPro signature evidence used in automatic assertion',
     'Reference': nan,
     'Comments': nan,
     'tag': ['SBW25', 'PFLU0001'],
     '_id': ObjectId('5e29981ed55d7e125759b030')},
    1: {'Ontology': 'Molecular Function',
     'Accession': 'GO:0003677',
     'Term': 'DNA binding',
     'GO Evidence': 'ISM Inferred from Sequence Model  Term mapped from: InterPro:PS01008',
     'Evidence Ontology (ECO) Code': 'ECO:0000259 match to InterPro signature evidence used in automatic assertion',
     'Reference': nan,
     'Comments': nan,
     'tag': ['SBW25', 'PFLU0001'],
     '_id': ObjectId('5e29981ed55d7e125759b031')},
    2: {'Ontology': 'Biological Process',
     '

In [ ]:
sbw25_pdc.list_collection_names()

In [216]:
doc = sbw25_pdc.features.find(filter={"Locus_Tag": {'$regex':'PFLU003[0-9]'}}, projection={"Locus_Tag"})

In [217]:
doc.count()

/opt/anaconda3/envs/sbw25/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


20

In [218]:
[d for d in doc]

[{'_id': ObjectId('5e286e0e34a983cbe88c0072'), 'Locus_Tag': 'PFLU0030'},
 {'_id': ObjectId('5e286e1334a983cbe88c0078'), 'Locus_Tag': 'PFLU0031'},
 {'_id': ObjectId('5e286e1934a983cbe88c0080'), 'Locus_Tag': 'PFLU0032'},
 {'_id': ObjectId('5e286e1e34a983cbe88c0090'), 'Locus_Tag': 'PFLU0033'},
 {'_id': ObjectId('5e286e2534a983cbe88c009d'), 'Locus_Tag': 'PFLU0034'},
 {'_id': ObjectId('5e286e2a34a983cbe88c00a2'), 'Locus_Tag': 'PFLU0035'},
 {'_id': ObjectId('5e286e3034a983cbe88c00ad'), 'Locus_Tag': 'PFLU0036'},
 {'_id': ObjectId('5e286e3534a983cbe88c00ba'), 'Locus_Tag': 'PFLU0037'},
 {'_id': ObjectId('5e286e3b34a983cbe88c00c9'), 'Locus_Tag': 'PFLU0038'},
 {'_id': ObjectId('5e286e4034a983cbe88c00cd'), 'Locus_Tag': 'PFLU0039'},
 {'_id': ObjectId('5e286eeb34a983cbe88c0157'), 'Locus_Tag': 'PFLU0030'},
 {'_id': ObjectId('5e286ef134a983cbe88c015d'), 'Locus_Tag': 'PFLU0031'},
 {'_id': ObjectId('5e286ef634a983cbe88c0165'), 'Locus_Tag': 'PFLU0032'},
 {'_id': ObjectId('5e286efb34a983cbe88c0175'), 'Loc